#An Analysis of Game of Thrones Battles

I want to do some analysis on the battles listed in **battles.csv** and depending on the results, I'd like to create some predictions.

This is my second ever Kaggle submission, so please give me feedback!


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt 
import seaborn as sns

# Input data files are available in the "../input/" directory.
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Set parameters on plt
plt.rcParams["axes.labelsize"] = 16.
plt.rcParams["xtick.labelsize"] = 13.
plt.rcParams["ytick.labelsize"] = 13.
plt.rcParams["legend.fontsize"] = 11.

# Read in files
battles = pd.read_csv("../input/battles.csv")
chardeaths = pd.read_csv("../input/character-deaths.csv")
charpreds = pd.read_csv("../input/character-predictions.csv")

# Does the army size have anything to do with victory?
I'm going to introduce two new variables here:

**numdefenders** = number of major houses defending in a battle

**numattackers** = number of major hours attacking in a battle

In [ ]:
battles.loc[:, "numdefenders"] = (4 - battles[["defender_1", "defender_2", "defender_3", "defender_4"]].isnull().sum(axis = 1))
battles.loc[:, "numattackers"] = (4 - battles[["attacker_1", "attacker_2", "attacker_3", "attacker_4"]].isnull().sum(axis = 1))

housesinvolved = battles.groupby('battle_number').sum()[["numdefenders", "numattackers"]].plot(kind = 'bar', figsize = (15,6), rot = 0)
_ = housesinvolved.set_xlabel("Battle Number"), housesinvolved.set_ylabel("Number of Houses Involved"), housesinvolved.legend(["Defenders", "Attackers"])


As we can see, there are a couple battles where there are four attackers, Battles 31 and 38, which are the Retaking of Deepwood Motte and the Siege of Winterfell. Before we delve further into that, I am going to create a few more graphs and variables.

In [ ]:
deathcapture = battles.groupby('battle_number').sum()[["major_death", "major_capture"]].plot(kind = 'bar', figsize = (15, 6), rot = 0)
_ = deathcapture.set_xlabel("Battle Number"), deathcapture.set_ylabel("Number of Major Deaths of Captures"), deathcapture.legend(["Major Deaths", "Major Captures"])

The graph above shows the number of major events that occured in each battle. From the graph we can infer that they did not happen very often. In fact, only six battles had both a major death and a major capture.

In [ ]:
d1 = battles.dropna(axis = 0, subset = [["attacker_size", "defender_size", "attacker_outcome"]]).copy(deep = True)
col = [sns.color_palette()[1] if x == "win" else "lightgray" for x in d1.attacker_outcome.values]
plot1 = d1.plot(kind = "scatter", x = "attacker_size", y = "defender_size", c = col, figsize = (15, 6), s = 100, lw = 2.)
_ = plot1.set_xlabel("Attacking Size"), plot1.set_ylabel("Defending Size"), plot1.legend("win")

From this graph, I surprisingly see no clear correlation or causation of the size of the attacking army being related to them being victorius. 

#Is there a correlation or a causation between the size of the battle and the occurrence of major deaths?

There does seem to be a correlation between the size of the battle and the occurrence of a major death, albeit a little iffy. In the two data points that have the largest defending army size and the single point that has an attacking army size of 100,000, a major death occurs. Below that, it seems the occurrence of a major death is slightly less likely and so on and so forth.

Thinking purely about entertainment value, this makes sense. The bigger an event, the more likely something will happen during that event.

In [ ]:
d2 = battles.dropna(axis=0, subset=[["attacker_size", "defender_size", "major_death"]]).copy(deep=True)
col = [sns.color_palette()[0] if x == 1 else sns.color_palette()[4] for x in d2.major_death.values]
plot2 = d2.plot(kind="scatter", x = "attacker_size", y = "defender_size", c = col, figsize=(15, 6), s = 100, lw = 2.)
plot2.set_xlabel("Attacking Size"), plot2.set_ylabel("Defending Size"), plot2.legend("Major Death")



In [ ]:
d3 = battles.groupby('region').sum()[["attacker_size", "defender_size"]].plot(kind='barh', figsize=(15,6), rot=0)
_ = d3.set_xlabel("Army Size"), d3.set_ylabel("Region"), d3.legend(["Attackers", "Defenders"])

The graph above has summed up every attacking army and defending army size according to the region the battles took place. THis would be useful coupled with the graph above, to determine the average attacking or defending army size for every region.

In [ ]:
d4 = battles['region'].value_counts().plot(kind='barh', figsize=(15,6))
_ = d4.set_xlabel("Number of Battles"), d4.set_ylabel("Regions")


In [ ]:
#Define the variables
battleregion = battles['region'].value_counts()
attacksize = battles.groupby('region').sum()[["attacker_size"]]
defendsize = battles.groupby('region').sum()[["defender_size"]]

#Find the average per region
avgattack = attacksize / battleregion
avgdefend = defendsize / battleregion

#Plot it
d5 = mpl.plt(avgattack.sort_values('region'), avgdefend.sort_values('region'), figsize=(15,6), rot=0)